In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from smart_open import smart_open
# nltk.download('stopwords')  # run once
from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
from gensim import corpora
from pprint import pprint
from pyclustering.cluster.kmedians import kmedians
from pyclustering.cluster import cluster_visualizer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES
# from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook',
#              ticks=True, grid=False)
import math
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
import operator
import pandas as pd
# jtplot.style(theme='monokai', context='notebook',
#              ticks=True, grid=False)

In [2]:
df= pd.read_csv("final_clustering_df.csv")
X = np.asarray(df.iloc[:,[1,2]])

In [3]:
def euclidean_distance_word(dat, rand_medians,word_sim):
    dist_list = []
    for i in range(len(rand_medians)):
        summation = np.sum((X-rand_medians[i])**2, axis=1)
        summation = normalize(summation.reshape(-1,1))
        summation = summation + (1-word_sim).reshape(-1,1)
#         summation = word_sim
        dist_list.append(np.sqrt(summation))
    dist_list = np.asarray(dist_list)
    return (dist_list)

def word_sim_calc(df,item):
    file_docs = []
    for i in range(len(df)):
        tokens = sent_tokenize(df['Names'][i])
        file_docs.append(tokens)
    mydict = corpora.Dictionary([simple_preprocess((" ".join(line))) for line in file_docs])
    corpus = [mydict.doc2bow(simple_preprocess((" ".join(line)))) for line in file_docs]
    tfidf = gensim.models.TfidfModel(corpus)
    workdir = r'C:\Users\akhil\OneDrive - University of Oklahoma\Subjects\Data Mining\Project\ '
    sims = gensim.similarities.Similarity(workdir,tfidf[corpus],num_features=len(mydict))
    query_doc_tf_idf = tfidf[corpus[item]]
    word_sim = sims[query_doc_tf_idf]
    return (word_sim)\

def similar_prod(item,no_item,data):
    user = []
    X = np.asarray(data.iloc[:,[1,2]])
    user.append(X[item])
    wordsim = word_sim_calc(data,item)
    dist_array = euclidean_distance_word(X,user,wordsim)
    dict = { i : dist_array[:,i] for i in range(0, dist_array.shape[1] ) }
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1))
    similar_prod_index = sorted_dict[0:(no_item+1)]
    result = df.iloc[np.asarray(similar_prod_index)[:,0]]
    name = result['Names'].tolist()
    price = result['Price'].tolist()
    rating = result['Rating'].tolist()
    return name,price,rating

In [11]:
ind = int(input("Select product index : ") )
num = int(input("Select no of similar products : ") )
similar_prod(ind,num,df)

Select product index : 0
Select no of similar products : 5


(['Power Rangers Megaforce Action Figure (Pink)',
  'Power Rangers Megaforce Vrak Action Figure',
  'Power Rangers Megaforce Esa Brak Action Figure',
  'Power Rangers Mega Force Action Figure (Metallic Blue)',
  'Power Rangers Super Mega Force 12.5 cm Action Figure (Red)',
  'Disney Frozen John Adams Blopens Activity Set'],
 [8.99, 8.99, 8.99, 8.99, 8.99, 8.99],
 [3.3, 3.3, 3.3, 3.3, 3.3, 3.3])

In [10]:
df.iloc[0,]

Names     Power Rangers Megaforce Action Figure (Pink)
Price                                             8.99
Rating                                             3.3
Name: 0, dtype: object